### Task 1: Validate Data with a Custom Expectation in Great Expectations
**Description**: Create a custom expectation and validate data with Great Expectations.

**Load a sample DataFrame**

data = {
'age': [25, 30, 35, 40, 45],
'income': [50000, 60000, 75000, None, 100000]
}

In [2]:
# Write your code from here
import great_expectations as ge
import pandas as pd
data={'age':[25,30,35,40,45],'income':[50000,60000,75000,None,100000]}
df=pd.DataFrame(data)
validator=ge.from_pandas(df)
validator.expect_column_values_to_not_be_null(column="income")
validator.expect_column_values_to_be_between(column="age",min_value=18,max_value=100)
results=validator.validate()
print(results)


AttributeError: module 'great_expectations' has no attribute 'from_pandas'

### Task 2: Implement a Basic Alert System for Data Quality Drops
**Description**: Set up a basic alert system that triggers when data quality drops.

In [ ]:
# Write your code from here
import pandas as pd
from great_expectations.data_context import FileDataContext
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.core.yaml_handler import YAMLHandler
import os

# Import your custom expectation.
# This MUST be done *before* loading the context, so GX knows about it.
from my_custom_expectations import ExpectColumnValuesToBeDivisibleBy

# --- Configuration ---
csv_file_path = 'data.csv' # Ensure this file exists
expectation_suite_name = "my_sample_dataframe_suite"
checkpoint_name = "my_daily_data_quality_check"

# --- Sample DataFrame (same as before) ---
data = {
    'age': [25, 30, 35, 40, 45, 27, None],
    'income': [50000, 60000, 75000, None, 100000, 55000, 80000]
}
df = pd.DataFrame(data)

# --- Great Expectations Workflow ---

# 1. Load the Data Context
context = FileDataContext(project_root_dir='.')

# Manually add custom expectation for this session if not configured in great_expectations.yml
context.add_custom_expectation(ExpectColumnValuesToBeDivisibleBy, "column_values.divisible_by")

# 2. Create a Batch Request using your DataFrame
batch_request = RuntimeBatchRequest(
    datasource_name="my_pandas_datasource",
    data_connector_name="default_runtime_data_connector",
    data_asset_name="my_data_asset",
    batch_spec_passthrough={"batch_data": df},
)

# 3. Get a Validator and define Expectations
print(f"Loading/Creating expectation suite: {expectation_suite_name}")
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
    create_expectation_suite_if_absent=True
)

# Clear existing expectations to redefine for this run if needed, or comment out
# validator.get_expectation_suite().clear()

print("\nDefining expectations...")
# Example 1: Custom expectation - likely to pass for most values
validator.expect_column_values_to_be_divisible_by(column="age", divisor=5, meta={"notes": "Expected to have some non-divisible values due to '27' and 'None'"})

# Example 2: Custom expectation - likely to fail significantly
validator.expect_column_values_to_be_divisible_by(column="age", divisor=10, meta={"notes": "Expected to fail for non-multiples of 10 and nulls"})

# Example 3: Standard GE expectation - will fail due to None
validator.expect_column_values_to_not_be_null(column="age", meta={"notes": "Expected to fail due to null value in age"})

# Example 4: Another standard GE expectation - will fail due to None
validator.expect_column_values_to_not_be_null(column="income", meta={"notes": "Expected to fail due to null value in income"})

# 5. Save the Expectation Suite
print("\nSaving expectation suite...")
validator.save_expectation_suite(discard_failed_expectations=False)

# 6. Run the Validation using a Checkpoint
# If you haven't defined this checkpoint in great_expectations/checkpoints/your_checkpoint_name.yml,
# it will use an ephemeral one based on the parameters passed.
# For a more persistent setup, you'd define the checkpoint in YAML.
print(f"\nRunning checkpoint: {checkpoint_name}")
checkpoint_result = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
    # This configuration defines what actions to take after validation,
    # including building Data Docs and optionally sending notifications.
    # For a real alert system, you'd configure actions here.
    action_list=[
        {
            "name": "store_validation_result",
            "action": {"class_name": "StoreValidationResultAction"},
        },
        {
            "name": "store_evaluation_parameter_metrics",
            "action": {"class_name": "StoreEvaluationParameterMetricsAction"},
        },
        {
            "name": "update_data_docs",
            "action": {"class_name": "UpdateDataDocsAction"},
        }
    ]
)

# --- Step 7: Implement Basic Alert System ---
print("\n--- Basic Alert System Check ---")

if not checkpoint_result.success:
    print("🚨 ALERT! Data Quality Drop Detected! 🚨")
    print(f"Checkpoint '{checkpoint_name}' FAILED for suite '{expectation_suite_name}'.")
    print("Review the Data Docs for details on failed expectations.")

    # You could add more sophisticated alerting here:
    # 1. Send an email:
    #    import smtplib
    #    from email.mime.text import MIMEText
    #    msg = MIMEText(f"Data quality check failed for {expectation_suite_name}. See Data Docs for more info.")
    #    msg['Subject'] = 'Data Quality Alert!'
    #    msg['From'] = 'your_email@example.com'
    #    msg['To'] = 'alert_recipient@example.com'
    #    with smtplib.SMTP('smtp.example.com', 587) as s:
    #        s.starttls()
    #        s.login('your_email@example.com', 'your_password')
    #        s.send_message(msg)
    #    print("Email alert sent!")

    # 2. Post to Slack/Teams (requires webhooks or SDK)
    #    import requests
    #    slack_webhook_url = "YOUR_SLACK_WEBHOOK_URL"
    #    payload = {"text": f"🚨 Data Quality Alert! Checkpoint '{checkpoint_name}' failed for suite '{expectation_suite_name}'."}
    #    response = requests.post(slack_webhook_url, json=payload)
    #    if response.status_code == 200:
    #        print("Slack alert sent!")
    #    else:
    #        print(f"Failed to send Slack alert: {response.status_code} - {response.text}")

    # 3. Log to a monitoring system
    # import logging
    # logging.basicConfig(level=logging.INFO)
    # logging.info(f"DATA_QUALITY_ALERT: Checkpoint {checkpoint_name} failed. Suite: {expectation_suite_name}")

else:
    print("✅ Data Quality is good! All expectations passed. ✅")

# 8. Build and Open Data Docs (Optional, but highly recommended for viewing results)
print("\nBuilding Data Docs. Check your browser for the validation report.")
context.build_data_docs()
context.open_data_docs()

print("\nValidation and Alerting process complete.")

ModuleNotFoundError: No module named 'my_custom_expectations'

### Task 3: Real-time Data Quality Monitoring with Python and Great Expectations
**Description**: Implement a system that monitors data quality in real-time.

In [ ]:
# Write your code from here